In [1]:
import gsd.hoomd
import numpy as np

In [2]:
def compute_all_flake_coms(flake_positions, box_lengths, beads_per_flake=50):
    Lx, Ly, Lz = box_lengths
    n_flakes = len(flake_positions) // beads_per_flake
    flake_coms = []

    for i in range(n_flakes):
        flake = flake_positions[i * beads_per_flake : (i + 1) * beads_per_flake]
        ref = flake[0]
        delta = flake - ref

        # Minimum image convention
        delta[:, 0] -= Lx * np.round(delta[:, 0] / Lx)
        delta[:, 1] -= Ly * np.round(delta[:, 1] / Ly)
        delta[:, 2] -= Lz * np.round(delta[:, 2] / Lz)

        unwrapped = ref + delta
        com = np.mean(unwrapped, axis=0)
        flake_coms.append(com)

    return np.array(flake_coms)

In [3]:
def find_flake_neighbors(coms, box_lengths, cutoff):
    Lx, Ly, Lz = box_lengths
    n = len(coms)
    for i in range(n):
        neighbors = []
        for j in range(n):
            if i == j:
                continue
            dr = coms[j] - coms[i]
            dr[0] -= Lx * np.round(dr[0] / Lx)
            dr[1] -= Ly * np.round(dr[1] / Ly)
            dr[2] -= Lz * np.round(dr[2] / Lz)
            dist = np.linalg.norm(dr)
            if dist < cutoff:
                neighbors.append((j, dist))
        if neighbors:
            neighbor_str = ", ".join([f"Flake {j} (distance: {d:.2f})" for j, d in neighbors])
            print(f"Flake {i} is neighbors with: {neighbor_str}")
        else:
            print(f"Flake {i} has no neighbors within cutoff range.")

In [4]:
traj = gsd.hoomd.open("100_10mer10f_0.0005dt_7kT_large.gsd")
frame = traj[93220]  # frame index

In [5]:
flake_type = "F"
flake_typeid = frame.particles.types.index(flake_type)
positions = frame.particles.position
typeids = frame.particles.typeid
flake_positions = positions[typeids == flake_typeid]

Lx, Ly, Lz = frame.configuration.box[:3]

flake_coms = compute_all_flake_coms(flake_positions, (Lx, Ly, Lz), beads_per_flake=50)
find_flake_neighbors(flake_coms, (Lx, Ly, Lz), cutoff=2.5)

Flake 0 has no neighbors within cutoff range.
Flake 1 has no neighbors within cutoff range.
Flake 2 is neighbors with: Flake 6 (distance: 1.15), Flake 8 (distance: 2.27)
Flake 3 has no neighbors within cutoff range.
Flake 4 has no neighbors within cutoff range.
Flake 5 has no neighbors within cutoff range.
Flake 6 is neighbors with: Flake 2 (distance: 1.15)
Flake 7 has no neighbors within cutoff range.
Flake 8 is neighbors with: Flake 2 (distance: 2.27), Flake 9 (distance: 2.17)
Flake 9 is neighbors with: Flake 8 (distance: 2.17)
